! alle Pakete installieren

# Pakete importieren 

In [ ]:
import random
import statistics

# Code

## 1) Parameter des Systems

In [ ]:
LAMBDA_ANKUNFT_PRO_WOCHE = 20   # Ø neue Anmeldungen pro Woche
SIMULATIONS_DAUER_WOCHEN = 200  # über wie viele Wochen simuliert wird

# Kapazität
ANZAHL_THERAPEUTEN = 5          # Anzahl Therapeut*innen

# Therapiedauer (Servicezeit) in Wochen
MIN_THERAPIEDAUER = 10          # z. B. 10 Sitzungen à 1 Woche
MAX_THERAPIEDAUER = 40          # z. B. 40 Sitzungen

# Zufalls-Seed für Reproduzierbarkeit (ändern(?))
random.seed(42)

## 2) Funktionen

In [ ]:
def generiere_ankunftszeiten(lambda_pro_woche, sim_dauer_wochen):
    """
    Erzeugt eine Liste aufsteigender Ankunftszeiten (in Wochen)
    auf Basis eines Poisson-Prozesses (exponentielle Zwischenankunftszeiten).
    """
    ankunftszeiten = []
    aktuelle_zeit = 0.0

    # mittlere Zwischenankunftszeit
    mean_interarrival = 1.0 / lambda_pro_woche  # Wochen

    while aktuelle_zeit < sim_dauer_wochen:
        # Exponentiell verteilte Zwischenankunftszeit
        delta = random.expovariate(1.0 / mean_interarrival)
        aktuelle_zeit += delta
        if aktuelle_zeit > sim_dauer_wochen:
            break
        ankunftszeiten.append(aktuelle_zeit)

    return ankunftszeiten


def ziehe_therapiedauer(min_wochen, max_wochen):
    """
    Ziehe eine zufällige Therapiedauer (in Wochen).
    Hier: einfache Gleichverteilung zwischen min und max.
    Später kannst du z.B. Normal- oder andere Verteilungen verwenden.
    """
    return random.uniform(min_wochen, max_wochen)


## 3) Zielgrößen

## 4) Simulationslauf

In [ ]:
def simuliere_warteschlange():
    # 3.1 Ankunftszeiten generieren
    ankunftszeiten = generiere_ankunftszeiten(
        LAMBDA_ANKUNFT_PRO_WOCHE,
        SIMULATIONS_DAUER_WOCHEN
    )

    # nächster Zeitpunkt, an dem jede/r Therapeut*in frei ist
    # Index: Therapeut, Wert: Zeit (in Wochen)
    naechst_frei = [0.0 for _ in range(ANZAHL_THERAPEUTEN)]

    # Listen zur Auswertung
    wartezeiten = []     # nur echte Wartezeit (bis Therapiebeginn)
    systemzeiten = []    # Zeit von Anmeldung bis Therapieende

    # zur Auslastungsberechnung: gesamte Behandlungszeit je Therapeut
    gesamt_behandlungszeit_pro_therapeut = [0.0 for _ in range(ANZAHL_THERAPEUTEN)]

    for ankunft in ankunftszeiten:
        # Therapiedauer für diesen Patienten
        servicezeit = ziehe_therapiedauer(
            MIN_THERAPIEDAUER, MAX_THERAPIEDAUER
        )

        # 3.2 Finde den Therapeuten, der am frühesten frei ist
        therapeut_index = min(range(ANZAHL_THERAPEUTEN),
                              key=lambda i: naechst_frei[i])
        frueheste_frei_zeit = naechst_frei[therapeut_index]

        # 3.3 Therapiebeginn: max(Ankunft, frühester Frei-Zeitpunkt)
        therapiebeginn = max(ankunft, frueheste_frei_zeit)
        therapieende = therapiebeginn + servicezeit

        # 3.4 Warte- und Systemzeit berechnen
        wartezeit = therapiebeginn - ankunft
        systemzeit = therapieende - ankunft

        wartezeiten.append(wartezeit)
        systemzeiten.append(systemzeit)

        # 3.5 Kalender des Therapeuten aktualisieren
        naechst_frei[therapeut_index] = therapieende
        gesamt_behandlungszeit_pro_therapeut[therapeut_index] += servicezeit

    # ============================================
    # 4. Kennzahlen berechnen
    # ============================================

    # Durchschnittliche Wartezeit & Systemzeit
    mittlere_wartezeit = statistics.mean(wartezeiten) if wartezeiten else 0.0
    mittlere_systemzeit = statistics.mean(systemzeiten) if systemzeiten else 0.0

    # Median & ein paar Quantile (optional)
    wartezeiten_sortiert = sorted(wartezeiten)
    def quantil(values, q):
        if not values:
            return 0.0
        idx = int(q * (len(values) - 1))
        return values[idx]

    median_warte = quantil(wartezeiten_sortiert, 0.5)
    q75_warte = quantil(wartezeiten_sortiert, 0.75)
    q90_warte = quantil(wartezeiten_sortiert, 0.90)

    # Auslastung der Therapeut*innen
    # Annahme: Simulation läuft SIMULATIONS_DAUER_WOCHEN,
    # und jede/r Therapeut*in könnte theoretisch durchgehend arbeiten.
    auslastungen = [
        behandlungszeit / SIMULATIONS_DAUER_WOCHEN
        for behandlungszeit in gesamt_behandlungszeit_pro_therapeut
    ]
    mittlere_auslastung = statistics.mean(auslastungen) if auslastungen else 0.0

    # Anteil mit Wartezeit > X Wochen (z. B. 12 und 24 Wochen)
    def anteil_mit_wartezeit_groesser_x(x):
        if not wartezeiten:
            return 0.0
        return sum(w > x for w in wartezeiten) / len(wartezeiten)

    anteil_ueber_12 = anteil_mit_wartezeit_groesser_x(12)
    anteil_ueber_24 = anteil_mit_wartezeit_groesser_x(24)

## 5) Kennzahlen berechnen 

In [ ]:
# Durchschnittliche Wartezeit & Systemzeit
    mittlere_wartezeit = statistics.mean(wartezeiten) if wartezeiten else 0.0
    mittlere_systemzeit = statistics.mean(systemzeiten) if systemzeiten else 0.0

    # Median & ein paar Quantile (optional)
    wartezeiten_sortiert = sorted(wartezeiten)
    def quantil(values, q):
        if not values:
            return 0.0
        idx = int(q * (len(values) - 1))
        return values[idx]

    median_warte = quantil(wartezeiten_sortiert, 0.5)
    q75_warte = quantil(wartezeiten_sortiert, 0.75)
    q90_warte = quantil(wartezeiten_sortiert, 0.90)

    # Auslastung der Therapeut*innen
    # Annahme: Simulation läuft SIMULATIONS_DAUER_WOCHEN,
    # und jede/r Therapeut*in könnte theoretisch durchgehend arbeiten.
    auslastungen = [
        behandlungszeit / SIMULATIONS_DAUER_WOCHEN
        for behandlungszeit in gesamt_behandlungszeit_pro_therapeut
    ]
    mittlere_auslastung = statistics.mean(auslastungen) if auslastungen else 0.0

    # Anteil mit Wartezeit > X Wochen (z. B. 12 und 24 Wochen)
    def anteil_mit_wartezeit_groesser_x(x):
        if not wartezeiten:
            return 0.0
        return sum(w > x for w in wartezeiten) / len(wartezeiten)

    anteil_ueber_12 = anteil_mit_wartezeit_groesser_x(12)
    anteil_ueber_24 = anteil_mit_wartezeit_groesser_x(24)

## 6) Ergbenisse ausgeben

In [ ]:
print("=== Ergebnisse der Simulation (Therapie-Warteschlange) ===")
    print(f"Anzahl Therapeut*innen:            {ANZAHL_THERAPEUTEN}")
    print(f"Simulierte Zeit:                   {SIMULATIONS_DAUER_WOCHEN:.1f} Wochen")
    print(f"Anzahl Anmeldungen:                {len(ankunftszeiten)}")

    print("\nWartezeiten (in Wochen)")
    print(f"  Mittelwert:                      {mittlere_wartezeit:.2f}")
    print(f"  Median:                          {median_warte:.2f}")
    print(f"  75%-Quantil:                     {q75_warte:.2f}")
    print(f"  90%-Quantil:                     {q90_warte:.2f}")
    print(f"  Anteil > 12 Wochen:              {anteil_ueber_12*100:.1f} %")
    print(f"  Anteil > 24 Wochen:              {anteil_ueber_24*100:.1f} %")

    print("\nGesamtzeit im System (Anmeldung bis Therapieende)")
    print(f"  Mittelwert:                      {mittlere_systemzeit:.2f} Wochen")

    print("\nAuslastung der Therapeut*innen")
    for i, rho in enumerate(auslastungen):
        print(f"  Therapeut {i+1}:                 {rho*100:.1f} %")
    print(f"  Durchschnittliche Auslastung:    {mittlere_auslastung*100:.1f} %")
    
    if __name__ == "__main__":
    simuliere_warteschlange()

# Verbesserungen

## a)